In [43]:
# !pip install pandas seaborn matplotlib sklearn
# !pip install pandas 

In [44]:
import pandas as pd

In [45]:
train_images_path = "/home/docker_current/datasets/train"
df_train = pd.read_csv("/home/docker_current/datasets/train.csv")
df_train['text_features'] = None
df_train['image_features'] = None
df_train['pred_id'] = 0
df_train.head(3)

,id,description,object_img,text_features,image_features,pred_id
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,None,None,0
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,None,None,0
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,None,None,0


In [46]:
from multilingual_clip import Config_MCLIP
import transformers
import torch

class MultilingualCLIP(transformers.PreTrainedModel):
    config_class = Config_MCLIP.MCLIPConfig

    def __init__(self, config, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.transformer = transformers.AutoModel.from_pretrained(config.modelBase, cache_dir=kwargs.get("cache_dir"))
        self.LinearTransformation = torch.nn.Linear(in_features=config.transformerDimensions,
                                                    out_features=config.numDims)

    def forward(self, txt, tokenizer):
        txt_tok = tokenizer(txt, padding=True, return_tensors='pt', truncation=True)
        embs = self.transformer(**txt_tok)[0]
        att = txt_tok['attention_mask']
        embs = (embs * att.unsqueeze(2)).sum(dim=1) / att.sum(dim=1)[:, None]
        return self.LinearTransformation(embs)

    @classmethod
    def _load_state_dict_into_model(cls, model, state_dict, pretrained_model_name_or_path, _fast_init=True):
        model.load_state_dict(state_dict)
        return model, [], [], []

In [47]:
import transformers

import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"

def get_text_encode_model(model_name='M-CLIP/XLM-Roberta-Large-Vit-L-14'):
    model = MultilingualCLIP.from_pretrained(model_name)
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, model_max_length=512)
    return model, tokenizer

def get_image_encode_model(model_name="ViT-L/14"):
    model, preprocess = clip.load(model_name, device=device)
    return model, preprocess 

In [48]:
text_model, text_tokenizer = get_text_encode_model()
image_model, image_preproc = get_image_encode_model()
None

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
def get_image_text_features(df, index_text, index_image):
    sample_text = df['description'][index_text]
    sample_image_path = "/home/docker_current/datasets/train/" + str(df['object_img'][index_image]) + ".png"
    # print(sample_text)
    # print(sample_image_path)

    image = Image.open(sample_image_path)
    image = image_preproc(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = image_model.encode_image(image).cpu().detach().numpy()

    text_features = text_model.forward(sample_text, text_tokenizer).cpu().detach().numpy()

    return image_features, text_features

In [50]:
%%time
#Добавляем в датафрейм фичи текста и изображений, чтоб потом с ними быстро работать
# for i in range(5):
for i in range(len(df_train)):
    image_features, text_features = get_image_text_features(df_train, i, i)
    df_train['text_features'][i] = text_features
    df_train['image_features'][i] = image_features

/home/docker_current/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/docker_current/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


CPU times: user 2h 32min 19s, sys: 2min 6s, total: 2h 34min 25s
Wall time: 24min 9s


In [72]:
df_train['image_features'] = df_train['image_features'].apply(lambda x: list(x[0]))
df_train['text_features'] = df_train['text_features'].apply(lambda x: list(x[0]))
# df_train.to_csv('train_MLCLIP_feat.csv', index=False)

In [92]:
import numpy as np
image_feat_array = np.vstack((i for i in df_train['image_features']))
text_feat_array = np.vstack((i for i in df_train['text_features']))

with open('image_feat_array.npy', 'wb') as f:
    np.save(f, image_feat_array)

with open('text_feat_array.npy', 'wb') as f:
    np.save(f, text_feat_array)

/home/docker_current/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  
/home/docker_current/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
# %%time
# # from tqdm.notebook import tqdm

# cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

# sim = []

# for index_image in range(len(df_train)):
#     image_features, text_features = get_image_text_features(df_train, 0, index_image)
#     similarity = cos(image_features, text_features)
#     sim.append(similarity)



CPU times: user 722 µs, sys: 14 µs, total: 736 µs
Wall time: 439 µs


tensor([0.2738], grad_fn=<DivBackward0>)

## samples 

In [ ]:
# from multilingual_clip import pt_multilingual_clip
# import transformers

# texts = [
#     'Three blind horses listening to Mozart.',
#     'Älgen är skogens konung!',
#     'Wie leben Eisbären in der Antarktis?',
#     'Вы знали, что все белые медведи левши?'
# ]
# model_name = 'M-CLIP/XLM-Roberta-Large-Vit-L-14'

# # Load Model & Tokenizer
# model = pt_multilingual_clip.MultilingualCLIP.from_pretrained(model_name)
# tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# embeddings = model.forward(texts, tokenizer)
# print("Text features shape:", embeddings.shape)

In [ ]:
# import torch
# import clip
# import requests
# from PIL import Image

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-L/14", device=device)

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
# image = preprocess(image).unsqueeze(0).to(device)

# with torch.no_grad():
#     image_features = model.encode_image(image)

# print("Image features shape:", image_features.shape) 